In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [ ]:
import os, sys
sys.path.append(os.path.join(os.path.dirname(os.path.abspath(os.getcwd())), "lib/python"))

import numpy as np

from picopic.cfg import Cfg
from picopic.plot_builder import PlotBuilder
from picopic.h5_reader import H5Reader
from picopic.plain_reader import PlainReader

In [ ]:
##  configuration options
el_charge = 1.6e-19
rho_beam_scale = 1
config_file = '../PicoPIC.json'

cfg = Cfg(config_file)

# update shape to get custom shaped images
# useful to get just part of frame
# or get frames, which has smaller shape than full frame
size=[0, 0, cfg.geometry_grid[0], cfg.geometry_grid[1]]

timestamp=5e-12
show_grid=False
use_cache=False

cmap='terrain'

cmp_0 = 'E_r'
cmp_1 = 'E_z'
cmp_beam = 'density'
specie_beam = 'beam_electrons'

# color limits (WARNING: clim_estimation may works incorrectly)
clim_estimation = 1e5 # cfg.get_clim_estimation()
clim_0 = [-clim_estimation, clim_estimation]
clim_1 = [-clim_estimation, clim_estimation]
clim_beam = [0, clim_estimation]

image_interpolation = 'nearest'

autoselect = True

x_axis_label = r'$\mathit{Z (m)}$'
y_axis_label = r'$\mathit{R (m)}$'
cbar_axis_label = r'$\frac{V}{m}$'
cbar_bunch_density_axis_label = r'$m^{-3}$'

plot_name_0 = r'$\mathbf{Electric\enspace Field\enspace Radial \enspace Component\enspace (E_r)}$'
plot_name_1 = r'$\mathbf{Electric\enspace Field\enspace Longitudial \enspace Component\enspace (E_z)}$'
plot_name_beam = r'$\mathbf{Electrons\enspace Beam\enspace Density \enspace  (\rho_{beam})}$'

In [ ]:
# define reader (plain reader used)
if not cfg.use_hdf5:
    reader = PlainReader(path = cfg.data_path,
                         fullframe_size=cfg.geometry_grid,
                         fpds=1, 
                         use_cache=use_cache,
                         verbose=True)
else:
    reader = H5Reader(str(os.path.join(cfg.data_path, 'data.h5')), use_cache=use_cache)
    reader.verbose = True

r_scale = (size[2] - size[0]) / cfg.geometry_grid[0]
z_scale = (size[3] - size[1]) / cfg.geometry_grid[1]

In [ ]:
# get data
data_0 = []
data_1 = []
data_beam = []


for probe in cfg.probes:
    frame = cfg.get_frame_number_by_timestamp(timestamp, probe.schedule)
    if (probe.shape == 'rec') and (probe.size[0] == size[0]) and (probe.size[1] == size[1]) and(probe.size[2] == size[2]) and(probe.size[3] == size[3]):
        if probe.component == cmp_0: data_0 = reader.get_frame(cmp_0, size, frame)
        if probe.component == cmp_1: data_1 = reader.get_frame(cmp_1, size, frame)
        if probe.component == cmp_beam and probe.specie == specie_beam: data_beam = reader.get_frame("{}/{}".format(cmp_beam, specie_beam), size, frame)

# try bigger frames, if autoselect enabled
if (len(data_0) == 0 or len(data_1) == 0 or len(data_beam) == 0) and autoselect:
    for probe in cfg.probes:
        frame = cfg.get_frame_number_by_timestamp(timestamp, probe.schedule)
        if (probe.shape == 'rec') and (probe.size[0] <= size[0]) and (probe.size[1] <= size[1]) and(probe.size[2] >= size[2]) and(probe.size[3] >= size[3]):
            if probe.component == cmp_0 and len(data_0) == 0: data_0 = reader.get_frame("{}".format(cmp_0), probe.size, frame)[size[0]:size[2], size[1]:size[3]]
            if probe.component == cmp_1 and len(data_1) == 0: data_1 = reader.get_frame("{}".format(cmp_1), probe.size, frame)[size[0]:size[2], size[1]:size[3]]
            if probe.component == cmp_beam and probe.specie == specie_beam and len(data_beam) == 0: data_beam = reader.get_frame("{}/{}".format(cmp_beam, specie_beam), probe.size, frame)[size[0]:size[2], size[1]:size[3]]

In [ ]:
# define plot builder
plot = PlotBuilder(size[3] - size[1], size[2] - size[0],
                   fig_color=cfg.figure_color, 
                   fig_width=cfg.figure_width,
                   fig_height=cfg.figure_height, 
                   fig_dpi=cfg.figure_dpi,
                   font_family=cfg.figure_font_family,
                   font_name=cfg.figure_font_name,
                   font_size=cfg.figure_font_size,
                   
                   x_ticklabel_end=cfg.geometry_size[1] * z_scale, y_ticklabel_end=cfg.geometry_size[0] * r_scale,
                   tickbox=True, grid=show_grid, is_invert_y_axe=False,
                   aspect='equal', image_interpolation=image_interpolation, guess_number_ticks=20)

In [ ]:
# add subplots
plot.add_subplot_cartesian_2d(plot_name_0, 311, x_axe_label=x_axis_label, y_axe_label=y_axis_label)
plot.add_subplot_cartesian_2d(plot_name_1, 312, x_axe_label=x_axis_label, y_axe_label=y_axis_label)
plot.add_subplot_cartesian_2d(plot_name_beam, 313, x_axe_label=x_axis_label, y_axe_label=y_axis_label)

# add timestamp to plot
plot.get_figure().suptitle("Time: {:.2e} s".format(timestamp), x=.85, y=.95)

# add images
plot.add_image(plot_name_0, data_0, cmap=cmap, clim=clim_0)
plot.add_image(plot_name_1, data_1, cmap=cmap, clim=clim_1)
plot.add_image(plot_name_beam, data_beam, cmap=cmap, clim=clim_beam)

# add colorbars
plot.add_colorbar(plot_name_0, ticks=clim_0, title=cbar_axis_label)
plot.add_colorbar(plot_name_1, ticks=clim_1, title=cbar_axis_label)
plot.add_colorbar(plot_name_beam, ticks=clim_beam, title=cbar_axis_label)

plot.show()